<a href="https://colab.research.google.com/github/pascualcam/aiml_code/blob/main/ch6/ch6_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
import json

In [25]:
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
from bs4 import BeautifulSoup
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [27]:
data = [json.loads(line)
        for line in open("/content/drive/MyDrive/Colab Notebooks/data/sarcasm2.json", 'r', encoding='utf-8')]

# with open("/content/drive/MyDrive/Colab Notebooks/data/sarcasm2.json", 'r') as f:÷
#   datastore = json.load(f)

In [28]:
sentences = []
labels = []
urls = []
for item in data:
  sentence = item['headline'].lower()
  sentence = sentence.replace(",", " , ")
  sentence = sentence.replace(".", " , ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""

  for word in words:
    word = word.translate(table)
    if word not in stopwords:
      filtered_sentence = filtered_sentence + word + " "
  sentences.append(filtered_sentence)
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

<ipython-input-28-86c91bada00f>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [29]:
training_size = 2300
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = sentences[training_size:]

In [30]:
vocab_size = 20000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = "OOV"

In [31]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

In [32]:
word_index = tokenizer.word_index

In [33]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding='post')

print(word_index)

{'OOV': 1, 'new': 2, 'man': 3, 'trump': 4, 'not': 5, 'just': 6, 'report': 7, 'first': 8, 'can': 9, 'like': 10, 'one': 11, 'life': 12, 'house': 13, 'area': 14, 'year': 15, 'will': 16, 's': 17, 'says': 18, 'no': 19, 'day': 20, 'old': 21, 'get': 22, 'donald': 23, 'u': 24, 'woman': 25, 'time': 26, 'white': 27, 'now': 28, 'clinton': 29, 'obama': 30, 'people': 31, 'americans': 32, 'off': 33, 'study': 34, 'finds': 35, 'still': 36, 'back': 37, 'know': 38, 'say': 39, 'women': 40, 'show': 41, 'work': 42, 'make': 43, 'gop': 44, 'nation': 45, 'love': 46, 'president': 47, 'trumps': 48, 'gets': 49, '5': 50, 'bill': 51, 'years': 52, 'american': 53, '4': 54, 'party': 55, 'last': 56, 'parents': 57, 'video': 58, '10': 59, 'made': 60, 'makes': 61, 'mom': 62, 'watch': 63, 'see': 64, 'game': 65, 'best': 66, 'dead': 67, 'go': 68, 'way': 69, 'family': 70, 'cant': 71, 'big': 72, 'black': 73, 'world': 74, 'getting': 75, 'police': 76, 'child': 77, 'may': 78, 'million': 79, 'right': 80, '1': 81, 'hillary': 82, '

In [35]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [37]:
# tf.keras.layers.Embeddings(vocab_size, embedding_dim)

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])